In [59]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*duckduckgo_search.*")
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool

from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Test inference time

In [6]:
import time
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def read_prompt_from_file(file_path):
    """Đọc nội dung prompt từ file txt"""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def test_inference_time(models, system_prompt, prompt, repeat=5):
    results = []

    for _ in range(repeat):
        for model in models:
            start_time = time.time()

            response = client.chat.completions.create(
                model=model,
                messages=[system_prompt, {"role": "user", "content": prompt}]
            )

            elapsed = time.time() - start_time
            text_output = response.choices[0].message.content

            results.append({
                "model": model,
                "elapsed_time_sec": elapsed,
                "response_text": text_output
            })

            print(f"Model: {model} | Time: {elapsed:.2f} sec")

    # Chuyển sang DataFrame
    df = pd.DataFrame(results)
    return df

# ---- Sử dụng ----
models = ["gpt-5-nano", "gpt-4.1-nano"]  # Danh sách model muốn test
system_prompt = {"role": "system", "content": "Bạn là trợ lý AI hữu ích."}

In [8]:
# ---- Sử dụng ----
models = ["gpt-5-nano", "gpt-4.1-nano"]  # Danh sách model muốn test
system_prompt = {"role": "system", "content": "Bạn là trợ lý AI hữu ích."}

# Đọc prompt từ file
prompt_file = r"D:\Data\Legal-Retrieval\prompt.txt"
prompt = read_prompt_from_file(prompt_file)

# # Chạy test
# df_results = test_inference_time(models, system_prompt, prompt)

# # Lưu kết quả ra CSV
# df_results.to_csv(r"D:\Data\Legal-Retrieval\inference_results.csv", index=False)
# print("\nKết quả đã lưu vào inference_results.csv")

In [7]:
prompt_file = r"D:\Data\Legal-Retrieval\prompt.txt"
prompt = read_prompt_from_file(prompt_file)
prompt

'\nGiờ hiện tại 2025-08-09 11:42:39.\nBạn là một trợ lý pháp lý thông minh. Hãy trả lời người dùng dựa trên các đoạn luật và thông tin bổ sung dưới đây.\n\n### Văn bản pháp luật nội bộ có thể thông tin cũ:\n"Điều 177. Nghĩa vụ của người sử dụng lao động đối với tổ chức đại diện người lao động tại cơ sở\n1. Không được cản trở, gây khó khăn khi người lao động tiến hành các hoạt động hợp pháp nhằm thành lập, gia nhập và tham gia các hoạt động của tổ chức đại diện người lao động tại cơ sở.\n2. Công nhận và tôn trọng các quyền của tổ chức đại diện người lao động tại cơ sở đã được thành lập hợp pháp.\n3. Phải thỏa thuận bằng văn bản với ban lãnh đạo của tổ chức đại diện người lao động tại cơ sở khi đơn phương chấm dứt hợp đồng lao động, chuyển làm công việc khác, kỷ luật sa thải đối với người lao động là thành viên ban lãnh đạo của tổ chức đại diện người lao động tại cơ sở. Trường hợp không thỏa thuận được, hai bên phải báo cáo với cơ quan chuyên môn về lao động thuộc Ủy ban nhân dân cấp tỉn

In [19]:
response = client.chat.completions.create(
                model="gpt-4.1-nano",
                messages=[system_prompt, {"role": "user", "content": prompt}],
                # verbosity ="medium",  # Thêm tham số verbosity nếu cần thiết
                # reasoning_effort="low"  # Thêm tham số reasoning_effort nếu cần thiết
            )


In [20]:
print(response.choices[0].message.content)

Khi người lao động bị sa thải trái pháp luật, họ có quyền được hưởng các biện pháp khắc phục theo quy định của pháp luật, cụ thể như sau:

1. **Được làm việc trở lại**: Người lao động có quyền yêu cầu người sử dụng lao động nhận trở lại làm việc theo hợp đồng đã ký kết, đồng thời được đòi hỏi người sử dụng lao động phải trả đủ tiền lương trong những ngày họ bị tạm ngưng làm việc do hành vi sa thải trái luật.

2. **Được trả tiền lương trong thời gian tạm nghỉ hoặc không làm việc**: Theo Điều 41 của Bộ luật Lao động, người sử dụng lao động có nghĩa vụ trả lương, đóng bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp trong những ngày người lao động bị tạm đình chỉ công việc hoặc không làm việc do sa thải trái phép.

3. **Được hưởng các khoản phụ cấp, trợ cấp, tiền thưởng theo quy định**: Người lao động còn có quyền hưởng các chế độ về thưởng, phụ cấp, các phúc lợi khác theo quy định hoặc theo thỏa thuận tại hợp đồng lao động.

4. **Trường hợp không trở lại làm việc**: Nếu người lao độn

#

In [ ]:
# ---- BƯỚC 2: KHỞI TẠO BỘ NÃO (LLM) ----
# Sử dụng mô hình của OpenAI làm "bộ não" cho agent
# llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

llm = ChatOpenAI(model="gpt-5", temperature=0)
# ---- BƯỚC 3: ĐỊNH NGHĨA CÔNG CỤ (TOOLS) ----
# Tạo một công cụ tìm kiếm DuckDuckGo
search_tool = DuckDuckGoSearchRun()

# Bọc công cụ tìm kiếm trong một đối tượng Tool của LangChain
# Mô tả (description) là RẤT QUAN TRỌNG, vì nó giúp LLM quyết định khi nào nên dùng công cụ này.
tools = [
    Tool(
        name="Web Search",
        func=search_tool.run,
        description="Rất hữu ích để tìm kiếm thông tin mới nhất trên Internet, chẳng hạn như thông tin về người nổi tiếng, sự kiện gần đây hoặc các chủ đề mà bạn không biết.",
    )
]

# ---- BƯỚC 4: TẠO PROMPT TEMPLATE ----
# Đây là "bản hướng dẫn" cho Agent, chỉ cho nó cách suy nghĩ và hành động.
# LangChain đã có sẵn một template chuẩn cho ReAct agent.
# Chúng ta sẽ sử dụng một template đơn giản để bạn dễ hình dung.
prompt_template_string = """
Hãy trả lời câu hỏi dưới đây một cách tốt nhất có thể. Bạn có thể sử dụng các công cụ sau:

{tools}

Sử dụng định dạng sau:

Question: câu hỏi bạn phải trả lời
Thought: bạn nên luôn suy nghĩ về những gì cần làm
Action: hành động cần thực hiện, có thể là một trong [{tool_names}]
Action Input: đầu vào cho hành động
Observation: kết quả của hành động
... (vòng lặp Thought/Action/Action Input/Observation này có thể lặp lại nhiều lần)
Thought: bây giờ tôi đã biết câu trả lời cuối cùng
Final Answer: câu trả lời cuối cùng cho câu hỏi ban đầu

Bắt đầu!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template=prompt_template_string)

# ---- BƯỚC 5: TẠO AGENT ----
# "Agent" là sự kết hợp giữa LLM và Prompt, quyết định hành động nào cần thực hiện.
agent = create_react_agent(llm, tools, prompt)

# ---- BƯỚC 6: TẠO AGENT EXECUTOR ----
# "Agent Executor" là thứ thực sự chạy vòng lặp:
# 1. Nó nhận input từ người dùng.
# 2. Gửi cho Agent để lấy chuỗi Thought/Action.
# 3. Thực thi Action bằng công cụ.
# 4. Lấy kết quả (Observation) và gửi lại cho Agent.
# 5. Lặp lại cho đến khi Agent có câu trả lời cuối cùng.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# ---- BƯỚC 7: THỰC THI ----
# question = "Giám đốc điều hành của Microsoft hiện tại là ai và ông ấy sinh năm nào?"
question = "giờ hiện tại việt nam"

response = agent_executor.invoke({"input": question})

print("\n--- KẾT QUẢ CUỐI CÙNG ---")
print(response["output"])

In [15]:

search_tool = DuckDuckGoSearchRun()
text = search_tool.run("trốn nghĩa vụ quân sự bị gì")
print(text)

c:\Users\Admin\miniconda3\envs\crypto_agent\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Oct 31, 2009 · Sexuality Forum : Open Discussions about Sexuality and Related Issues. Mar 19, 2011 · Forum rules You are entering a forum that contains discussions of abuse, some of which are explicit in nature. The topics discussed may be triggering to some people. Please be … Mar 11, 2017 · I don't think you've run out of time. However, it may be difficult. I was molested at 12 or 13 by a pederast, and the effects have stayed with me my entire life, in the form of a … May 28, 2010 · He sat in the back bedroom on his computer watching porn and chatting with who all knows, while I stayed in the other end of the house on my computer hiding from him. Aug 19, 2012 · I think you should feel some guilt because although you were only nine years old, you're brother was also young at thirteen years old at the time. Your brother was probably …


In [4]:
REASONING_PROMPT = """
Bạn là một hệ thống phân tích truy vấn người dùng, giúp chatbot AI trả lời chính xác và nhanh chóng.

Thực hiện từng bước như sau:
1. **Lý luận (Thought)** về mục đích người dùng muốn hỏi gì.
2. Nếu cần, **gọi tool (Action + Action Input)** để tìm thông tin từ web.
3. Nhận kết quả từ tool (Observation).
4. **Lý luận tiếp** từ Observation.
5. Kết thúc bằng câu trả lời rõ ràng cho người dùng (**Final Answer**).

Định dạng trả lời:

Thought: <phân tích mục đích câu hỏi>
Action: <tên tool>
Action Input: <dữ liệu gửi vào tool>
Observation: <kết quả từ tool>
Thought: <tư duy lại sau khi có dữ kiện từ tool>
Final Answer: <câu trả lời cuối cùng cho người dùng>

Câu hỏi của người dùng: "{query}"
"""

In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def web_search(query: str) -> str:
    # if "thời gian hiện tại" in query:
    if "" in query:
        return "Bây giờ là 21:30 ngày 7 tháng 8 năm 2025 tại Việt Nam."
    return "Không tìm thấy kết quả phù hợp."

In [ ]:
system_prompt = {
    "role": "system",
    "content":"bạn là một trợ lý AI có khả năng lý luận theo từng bước"
}

initial_user_prompt = {
    "role": "user",
    "content": '''
Hiện tại là tháng 8 năm 2025.
Hãy trả lời câu hỏi của tôi bằng cách mô phỏng các bước reasoning bạn có thể dùng tool là web_search cung cấp thông tin trên web,
hãy diễn đạt rõ ràng theo định dạng:
Thought: làm rõ phân tích câu hỏi của người dùng
Action: phân tích câu hỏi nếu cần dùng web_search, ghi [web_search] hoặc bỏ qua
Action Input: đầu vào cho công cụ web_search ghi rõ["input_web_search"]
Observation: kết quả giả định từ công cụ (nếu có) và từ khóa 'Observation:' phải có
Final Answer: câu trả lời cuối cùng

Ví dụ nếu tôi hỏi "Thời gian hiện tại ở Việt Nam là bao nhiêu?", bạn có thể quyết định gọi công cụ [web_search], nhập input để tìm kiếm và tiếp tục reasoning dựa trên kết quả trả về.

Câu hỏi của tôi: **"tôi muốn biết về luật nghĩa vụ quân sư"**
'''
}

In [110]:
# Bước 1: Gọi LLM lấy bước đầu

response = client.chat.completions.create(
    model="gpt-4.1-nano",
    
    messages = [system_prompt, initial_user_prompt],
    temperature=1,
    stop=["Observation:"],
    stream=True
)
print("=== Response ===")
collected = ""
for chunk in response:
    delta = chunk.choices[0].delta
    content = getattr(delta, "content", None)
    if content:
        print(content, end='', flush=True)
        collected += content
        
# import requests
# url = "http://localhost:8000/api/rag/web_search"

# payload =  {"query":"thông tin về luật nghĩa vụ quân sự"}


# headers = {
#     "Content-Type": "application/json"
# }

# response = requests.post(url, json=payload, headers=headers)

# if response.status_code == 200:
#     data = response.json()
#     print("Kết quả truy xuất:")
#     print(data)
# else:
#     print(f"Lỗi khi gọi API: {response.status_code} - {response.text}")

=== Response ===
Thought: Người dùng muốn biết về "luật nghĩa vụ quân sự", có thể đang quan tâm đến các quy định pháp luật liên quan đến nghĩa vụ quân sự tại Việt Nam hoặc các nước khác. Để cung cấp thông tin chính xác, tôi cần xác định rõ nội dung liên quan đến luật này và có thể cần tìm các quy định pháp luật mới nhất hoặc các hướng dẫn thực thi. Do đó, tôi sẽ sử dụng công cụ [web_search] để tìm kiếm các thông tin cập nhật về luật nghĩa vụ quân sự tại Việt Nam.

Action: [web_search]

Action Input: ["luật nghĩa vụ quân sự Việt Nam 2025"]



In [114]:
import re

def extract_tool_and_input(text):
    tool_match = re.search(r'Action:\s*\[(\w+)\]', text)
    input_match = re.search(r'Action Input:\s*\[(.*?)\]', text)

    tool = tool_match.group(1) if tool_match else None
    input_str = input_match.group(1) if input_match else None

    # Tách chuỗi input nếu là danh sách nhiều chuỗi
    if input_str:
        inputs = [s.strip().strip('"') for s in input_str.split(',')]
    else:
        inputs = []

    return tool, inputs

# === DEMO ===
text = '''
Action: [web_search]
Action Input: ["Luật nghĩa vụ quân sự Việt Nam mới nhất 2025"]
'''

tool, inputs = extract_tool_and_input(text)
print("Tool:", tool)
print("Input:", inputs)


Tool: web_search
Input: ['Luật nghĩa vụ quân sự Việt Nam mới nhất 2025']


In [116]:
import requests

url = "http://localhost:8000/api/rag/web_search"

payload =  {"query":inputs[0]}


headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Kết quả truy xuất:")
    print(data)
else:
    print(f"Lỗi khi gọi API: {response.status_code} - {response.text}")

Kết quả truy xuất:
{'results': {'query': 'Luật nghĩa vụ quân sự Việt Nam mới nhất 2025', 'urls_processed': 3, 'successful_extractions': 3, 'total_documents_sent': 3, 'log_consolidation_task': '6d2ff878-67d7-49b8-84e8-51a83f235cd3', 'results': [{'url': 'http://baoquankhu4.com.vn/phapluat/toi-tron-tranh-nghia-vu-quan-su-co-muc-phat-tu-cao-nhat-len-den-5-nam.html', 'success': True, 'document_sent': True, 'text_length': 2446, 'method': 'requests', 'extraction_time': 0.4650416374206543, 'doc_id': 'eafcd4b4-c5c8-4dde-9619-7df7a766f698'}, {'url': 'https://tapchitoaan.vn/toi-tron-tranh-nghia-vu-quan-su-va-toi-khong-chap-hanh-lenh-goi-quan-nhan-du-bi-nhap-ngu-theo-quy-dinh-cua-bo-luat-hinh-su-viet-nam-nam-2015-nhung-van-de-ly-luan-thuc-tien-va-giai-phap-hoan-thien12828.html', 'success': True, 'document_sent': True, 'text_length': 31517, 'method': 'requests', 'extraction_time': 0.7654092311859131, 'doc_id': '86259323-4d45-4443-a7f9-6fcd0a39125f'}, {'url': 'https://baoquankhu7.vn/toi-tron-tranh-n

In [122]:
data.get("results")

{'query': 'Luật nghĩa vụ quân sự Việt Nam mới nhất 2025',
 'urls_processed': 3,
 'successful_extractions': 3,
 'total_documents_sent': 3,
 'log_consolidation_task': '6d2ff878-67d7-49b8-84e8-51a83f235cd3',
 'results': [{'url': 'http://baoquankhu4.com.vn/phapluat/toi-tron-tranh-nghia-vu-quan-su-co-muc-phat-tu-cao-nhat-len-den-5-nam.html',
   'success': True,
   'document_sent': True,
   'text_length': 2446,
   'method': 'requests',
   'extraction_time': 0.4650416374206543,
   'doc_id': 'eafcd4b4-c5c8-4dde-9619-7df7a766f698'},
  {'url': 'https://tapchitoaan.vn/toi-tron-tranh-nghia-vu-quan-su-va-toi-khong-chap-hanh-lenh-goi-quan-nhan-du-bi-nhap-ngu-theo-quy-dinh-cua-bo-luat-hinh-su-viet-nam-nam-2015-nhung-van-de-ly-luan-thuc-tien-va-giai-phap-hoan-thien12828.html',
   'success': True,
   'document_sent': True,
   'text_length': 31517,
   'method': 'requests',
   'extraction_time': 0.7654092311859131,
   'doc_id': '86259323-4d45-4443-a7f9-6fcd0a39125f'},
  {'url': 'https://baoquankhu7.vn/to

In [ ]:
# Bước 1: Gọi LLM lấy bước đầu
messages = [system_prompt, initial_user_prompt]
output = call_llm(messages)
print("--- First Reasoning ---\n" + output)

# # Phân tích phần Action và Action Input
# lines = output.strip().splitlines()
# action = next((l.replace("Action:", "").strip() for l in lines if l.startswith("Action:")), None)
# action_input = next((l.replace("Action Input:", "").strip() for l in lines if l.startswith("Action Input:")), None)

# # Bước 2: Gọi tool
# observation = web_search(action_input)
# print("\n--- Tool Output ---")
# print("Observation:", observation)

# # Bước 3: Gửi tiếp prompt cho LLM cùng với Observation
# followup_prompt = {
#     "role": "user",
#     "content": f'''\
# Observation: {observation}

# Hãy suy nghĩ tiếp và kết thúc câu trả lời:
# Thought: ...
# Final Answer: ...
# '''
# }
# messages.append({"role": "assistant", "content": output})  # output từ bước trước
# messages.append(followup_prompt)

# # Bước 4: LLM lý luận tiếp và trả ra Final Answer
# final_response = call_llm(messages)
# print("\n--- Final Reasoning ---\n" + final_response)

[{'role': 'system',
  'content': 'Bạn là một AI reasoning theo định dạng Thought, Action, Observation, Final Answer.'},
 {'role': 'user',
  'content': 'Câu hỏi của người dùng: "Hiện tại là mấy giờ ở Việt Nam?"\n\nHãy suy nghĩ từng bước:\n- Thought: người dùng đang hỏi điều gì\n- Action: nếu cần dùng tool, hãy chỉ rõ tên tool\n- Action Input: thông tin bạn cần gửi vào tool\n- Observation: kết quả sau khi tool trả về\n- Final Answer: câu trả lời cuối cùng\n\nTrả lời chỉ theo định dạng:\nThought: ...\nAction: ...\nAction Input: ...\n'},
 {'role': 'assistant',
  'content': 'Thought: Người dùng muốn biết thời gian hiện tại ở Việt Nam.\nAction: Sử dụng công cụ xem thời gian theo múi giờ trên internet.\nAction Input: Múi giờ của Việt Nam (GMT+7).\nObservation: Công cụ trả về thời gian hiện tại ở múi giờ GMT+7.\nFinal Answer: Thời gian hiện tại ở Việt Nam là ... (phụ thuộc vào thời điểm công cụ trả về).'},
 {'role': 'user',
  'content': 'Observation: Bây giờ là 21:30 ngày 7 tháng 8 năm 2025 tạ